In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## langchain tracing and tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [16]:
##ChatOpenAI is wrapper using you can call any open ai model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="o1-mini")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000298447CA9F0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000298447CAC60> root_client=<openai.OpenAI object at 0x000002984465FA10> root_async_client=<openai.AsyncOpenAI object at 0x0000029844D8FC50> model_name='o1-mini' temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [17]:
result = llm.invoke("what is agentic ai")
print(result.content)


**Agentic AI** refers to artificial intelligence systems designed with a degree of autonomy and the ability to take actions independently to achieve specific goals or objectives. The term "agentic" derives from "agency," which in this context means the capacity to act independently and make decisions. Agentic AI systems are often conceptualized as "agents" that perceive their environment, make decisions based on that perception, and execute actions to influence their environment towards desired outcomes.

### Key Characteristics of Agentic AI

1. **Autonomy**: Agentic AI systems can operate without constant human intervention. They can make decisions and take actions based on predefined objectives and learned experiences.

2. **Goal-Oriented Behavior**: These systems are designed to achieve specific goals or perform particular tasks. They prioritize actions that help them move closer to these objectives.

3. **Adaptability**: Agentic AI can learn from interactions with the environment,

In [22]:
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnableConfig

model=ChatGroq(model="gemma2-9b-it",groq_api_key="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") 

#response = model.invoke("Hello from VS Code")
print(model)
model.invoke("hi")


client=<groq.resources.chat.completions.Completions object at 0x0000029846BA0FE0> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000029846BA16D0> model_name='gemma2-9b-it' model_kwargs={} groq_api_key=SecretStr('**********')


APIConnectionError: Connection error.

## ChatPromptTemplate
This is useful for creating chatbot or question answering system
ChatPromptTemplate has 3 important message types
1-System Message : This is used to baseline AI assisant.If you want to set some behivour of assisant ,this is the place to do it.
2-HumanMessage: All user inputs are stored in it.
3-AIMessage: This stores response from LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        "system","You are an expert AI Engineer. Provide me answers based on the questions",
        "user","{input}"
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='system'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answers based on the questions'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='user'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [31]:
# chain

chain = prompt|llm

response = chain.invoke({"input":"can you tell me about langsmith?"})
print(response)
print(response.content)

content="Certainly! **LangSmith** is a comprehensive platform designed to enhance the development, deployment, and management of language model (LLM) applications. It plays a pivotal role in the lifecycle of AI-driven projects, especially those leveraging large language models like GPT-4. Here's an overview of what LangSmith offers and how it fits into the AI engineering ecosystem:\n\n### **1. Overview of LangSmith**\n\n- **Purpose:** LangSmith serves as a centralized hub for monitoring, debugging, and optimizing language model applications. It aims to streamline the workflow for AI engineers by providing tools that facilitate better performance tracking, error analysis, and continuous improvement of LLM-based systems.\n\n- **Integration:** It seamlessly integrates with popular frameworks and platforms such as LangChain, allowing developers to incorporate robust monitoring and management capabilities into their existing workflows.\n\n### **2. Key Features**\n\n- **Monitoring and Analyt

## StrOutputParse
OutputParser that parses LLMResult into the top likely string.


In [30]:
## stroutput Parser
from langchain_core.output_parsers import StrOutputParser
str_output_parser = StrOutputParser()

chain=prompt|llm|str_output_parser

response = chain.invoke({"input":"can you tell me about langsmith?"})
print(response)

As of my knowledge cutoff in October 2023, **LangSmith** is a comprehensive platform designed to assist developers, data scientists, and organizations in building, managing, and deploying language models effectively. Here's an overview of its key features and functionalities:

### **1. Model Development and Fine-Tuning**
LangSmith offers tools that simplify the process of developing and fine-tuning language models. Users can:

- **Customize Models:** Adjust various parameters and settings to tailor models to specific tasks or domains.
- **Integrate Datasets:** Easily import and manage datasets for training and fine-tuning purposes.
- **Experimentation:** Run multiple training experiments to compare model performance and optimize results.

### **2. Deployment and Integration**
Once models are trained, LangSmith facilitates their deployment into production environments:

- **Scalable Deployment:** Deploy models on scalable infrastructure to handle varying workloads efficiently.
- **API I

In [ ]:
## with pydantic
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# Define the LLM
llm = ChatOpenAI(temperature=0.7)

# Define desired output structure
class Joke(BaseModel):
    setup: str = Field(description="Question to set up a joke")
    punchline: str = Field(description="Answer to resolve the joke")

# Setup parser
parser = JsonOutputParser(pydantic_object=Joke)

# Setup prompt
prompt = PromptTemplate(
    template="Answer the user query\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Compose chain
chain = prompt | llm | parser

# Run chain
joke_query = "Tell me a joke"
result = chain.invoke({"query": joke_query})

print(result)


{'setup': "Why couldn't the bicycle stand up by itself?", 'punchline': 'It was two tired.'}


In [50]:
### Without Pydantic
joke_query = "Tell me a joke ."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

{'joke': "Why couldn't the bicycle stand up by itself? It was two tired!"}

### Assisgment:
Create a simple assistant that uses any LLM and should be pydantic, when we ask about any product it should give you two information product Name, product details tentative price in USD (integer). use chat Prompt Template.


In [ ]:
from pydantic import BaseModel,Field
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

# pydantic class
class ProductDetails(BaseModel):
    product_name:str=Field(description="Product name")
    product_details:str=Field(description="Details about product")
    Price:str=Field(description="Price of Product in $$")
# parser
output_parser = JsonOutputParser(pydantic_object = "ProductDetails")

# Prompt
prompt_template = PromptTemplate(
    template="Answer user query about product details.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# chain
chain = prompt_template | llm | output_parser

chain.invoke({"query" : "Please provide details of Vizio TV"})

{'brand': 'Vizio',
 'model': 'Smart TV',
 'screen_size': '55 inches',
 'resolution': '4K Ultra HD',
 'refresh_rate': '120Hz',
 'smart_features': ['Built-in WiFi', 'SmartCast', 'Voice Control'],
 'ports': ['HDMI', 'USB', 'Ethernet'],
 'additional_features': ['Dolby Vision HDR',
  'Full Array LED backlighting',
  'Works with Alexa and Google Assistant'],
 'price': '$699.99'}